In [ ]:
"""
This is a script to verify the camera observations of the real robot. We assume that you are using deoxys_vision (https://github.com/UT-Austin-RPL/deoxys_vision) for capturing images. If you are using a different vision pipeline, please modify the code accordingly. 
"""

import plotly.graph_objs as go

from deoxys import config_root
from deoxys.franka_interface import FrankaInterface
from deoxys.utils import YamlConfig
from deoxys.utils.input_utils import input2action
from deoxys.utils.io_devices import SpaceMouse
from deoxys.utils.log_utils import get_deoxys_example_logger

from deoxys_vision.networking.camera_redis_interface import \
    CameraRedisSubInterface
from deoxys_vision.utils.calibration_utils import load_default_extrinsics, load_default_intrinsics
from deoxys_vision.utils.camera_utils import assert_camera_ref_convention, get_camera_info

import sys
sys.path.append("..")
from orion.utils.misc_utils import depth_to_rgb
from orion.utils.o3d_utils import O3DPointCloud, scene_pcd_fn

from orion.utils.real_robot_utils import RealRobotObsProcessor

# Make sure that you've launched camera nodes somewhere else
observation_cfg = YamlConfig("../configs/real_robot_observation_cfg.yml").as_easydict()

observation_cfg.cameras = []
for camera_ref in observation_cfg.camera_refs:
    assert_camera_ref_convention(camera_ref)
    camera_info = get_camera_info(camera_ref)

    observation_cfg.cameras.append(camera_info)

obs_processor = RealRobotObsProcessor(observation_cfg,
                                        processor_name="ImageProcessor")
obs_processor.load_intrinsic_matrix(resize=False)
obs_processor.load_extrinsic_matrix()
extrinsic_matrix = obs_processor.get_extrinsic_matrix("agentview")
intrinsic_matrix = obs_processor.get_intrinsic_matrix("agentview")

color_imgs, depth_imgs = obs_processor.get_original_imgs()
print(color_imgs[0].shape)

pcd_points, pcd_colors = scene_pcd_fn(
    rgb_img_input=color_imgs[0],
    depth_img_input=depth_imgs[0],
    extrinsic_matrix=extrinsic_matrix,
    intrinsic_matrix=intrinsic_matrix,
)

print(pcd_points.shape)

In [ ]:
from orion.utils.misc_utils import plotly_draw_3d_pcd
plotly_draw_3d_pcd(pcd_points, pcd_colors)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(np.concatenate((color_imgs[0], depth_to_rgb(depth_imgs[0])), axis=1))
print(depth_imgs[0].min(), depth_imgs[0].max())